In [ ]:
import pandas as pd
import numpy as np


genre = pd.read_json('data/genre_gn_all.json', typ = 'series', encoding = "utf-8")
meta = pd.read_json('data/song_meta.json', typ = 'frame', encoding="utf-8")
train = pd.read_json('data/train.json', typ = 'frame',encoding="utf-8")
val = pd.read_json('data/val.json', typ = 'frame',encoding="utf-8")
test = pd.read_json('data/test.json', typ = 'frame',encoding="utf-8")

## 1. 날짜 거르기

에러가 있는 날짜 지우기

input : train , test, val // meta

output : meta 의 노래정보 수정..

#### 노래 발매일 > 플레이리스트 갱신일  많은 상위 10개의 플레이리스트

In [ ]:
song_date_info = dict(zip(meta.id.values, meta.issue_date.values))
test = train[['songs', 'updt_date']]
test.loc[:,'updt_date'] = [int(time.split(' ')[0].replace('-','')) for time in test['updt_date'].values]
res , idx = [], 0

for sgs,updt_date in test[['songs','updt_date']].values: 
    for s in sgs:
        if song_date_info[s] > updt_date:
            res.append( [idx, s] )
    idx += 1
del idx, song_date_info, test
print(pd.Series( [i[0] for i in res] ).value_counts().iloc[:10])
train.loc[ pd.Series( [i[0] for i in res] ).value_counts().iloc[:10].index  , :]

#### 노래 발매일 > 플레이리스트 갱신일  많은 상위 10개의 노래

In [ ]:
print(pd.Series( [i[1] for i in res] ).value_counts().iloc[:10])
meta.loc[ pd.Series( [i[1] for i in res] ).value_counts().iloc[:10].index  , :]
del res,sgs,updt_date,s

#### 처리

In [ ]:
def update_date( self ):

    # 노래 매핑
    map_df = self.complex_.loc[:, ['songs', 'updt_date']]

    ## time_delay..
    map_unnest = np.dstack((np.concatenate(map_df.songs.values), np.repeat(map_df.updt_date.values, list(map(len, map_df.songs)))))

    # 데이터 프레임으로 변환 후,
    map_df = pd.DataFrame(data=map_unnest[0], columns=map_df.columns)


    map_df['updt_date'] = [int(v[:10].replace("-", '')) for v in map_df.updt_date.values]
    date_df = map_df.groupby(['songs'])['updt_date'].agg({'max'}).reset_index().rename(columns={'max': 'updt_date'})
    date_df['songs'] = date_df['songs'].astype(int)

    # date_df 와 meta 속 song의 발행 날짜 merge
    meta_song = self.meta[['id', 'issue_date']].rename(columns={'id': 'songs'})
    date_df = date_df.merge(meta_song, how='outer', on='songs')

    #  meta 데이터의 노래더 느리면 노래를 플레이리스트 데이터 기준으로 바꿈
    date_df.loc[date_df['updt_date'] < date_df['issue_date'], 'issue_date'] = date_df.loc[date_df['updt_date'] < date_df['issue_date'], 'updt_date']

    # 메타데이터 변경
    self.meta["issue_date"] = date_df['issue_date'].astype(int)

    return self.meta

In [17]:
import pickle

with open( 'data/digitization/clu_song_emb_200.pickle' , 'rb' ) as f:
    data = pickle.load(f)
data.head()

,song_id,label,vector1,vector2,vector3,vector4,vector5,vector6,vector7,vector8,...,vector91,vector92,vector93,vector94,vector95,vector96,vector97,vector98,vector99,vector100
0,144663,81,-3.148425,0.750318,3.072613,0.525002,-3.512241,-3.017920,-1.046888,0.735382,...,-3.428068,-4.138845,-1.233291,-4.006755,1.873970,2.310007,3.472715,1.293818,4.058638,-0.218241
1,116573,60,-1.873080,-2.444498,1.273128,0.017727,1.029912,0.517884,-5.193015,-0.845168,...,-0.558408,-4.053355,1.670949,-3.107049,0.876867,-0.889317,3.208157,0.833461,2.511628,0.709390
2,357367,119,-2.241840,-1.554076,1.312914,0.359657,-0.988056,-0.062449,-0.063281,4.080726,...,0.204263,-1.627694,0.021887,-0.404910,-0.440495,-3.360089,2.566588,2.132551,2.838747,1.375059
3,366786,199,-2.515584,-1.300756,0.755813,-1.012893,-3.565576,-0.364969,-2.768528,-0.108413,...,-0.432053,-3.454705,3.130727,-2.727538,2.347215,-0.121203,4.718843,1.826789,2.781937,-1.202482
4,133143,158,-0.187791,3.788712,-2.405741,-3.098448,-1.652190,-3.954020,-2.370774,-5.517591,...,0.191713,-3.223214,3.268676,-1.586885,3.349843,3.537786,7.769712,1.660735,-0.974760,2.236546


In [2]:
data.sort_values('label')

,tv_complex,label,vector1,vector2,vector3,vector4,vector5,vector6,vector7,vector8,...,vector91,vector92,vector93,vector94,vector95,vector96,vector97,vector98,vector99,vector100
123770,Alena Cherny,0,0.010528,-0.033731,-0.017849,-0.002565,-0.008593,-0.012752,-0.046151,0.033980,...,-0.086932,0.061939,-0.025565,-0.078153,0.013095,-0.032909,0.011439,0.068875,-0.016653,-0.025496
85402,The Overlords,0,0.009195,0.001281,-0.000404,-0.000638,0.006942,0.003861,-0.004335,0.007552,...,-0.012129,0.011443,-0.005547,0.001588,-0.002997,-0.005145,-0.001783,0.020809,0.002841,-0.004116
85401,Microlin,0,0.018767,-0.004041,-0.004838,0.003070,0.001107,0.001642,-0.015968,0.012931,...,-0.026511,0.025405,-0.015752,-0.008472,0.000640,-0.013367,0.006058,0.029394,-0.003071,0.003607
85400,Terrasun,0,0.018440,-0.010974,-0.003561,-0.001226,0.003950,-0.012667,-0.012519,0.017992,...,-0.024878,0.021182,-0.018463,-0.014382,0.002481,-0.019511,0.003722,0.036105,0.002039,0.003974
85399,X-Dream,0,0.021967,-0.003152,-0.001158,0.000561,0.001916,0.000668,-0.013582,0.017366,...,-0.028403,0.018385,-0.012129,-0.013365,-0.000335,-0.021283,0.006414,0.032752,0.002570,0.001621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,샘김 (Sam Kim),99,-4.876300,6.086558,2.879975,1.641567,2.309562,-0.089675,-0.837761,1.994895,...,3.301278,-2.850363,0.672755,1.797362,4.149090,3.051738,-2.229614,4.857689,4.048566,-4.091033
270,The Quiett,99,-3.871353,4.258779,-1.507257,4.449578,3.867599,-1.689985,1.142217,1.213963,...,-0.529289,-3.440601,2.202000,3.149529,0.476649,4.522182,0.393172,0.974170,0.394058,-2.539556
615,Redfoo,99,3.803059,1.171238,0.044757,1.382291,5.135074,-4.345951,-0.461358,2.987708,...,2.269397,-0.478430,1.683666,3.579565,0.483815,2.056346,1.691965,0.293405,4.400105,-2.296814
638,Michael Carreon,99,-0.124901,3.364625,1.982814,2.964050,0.559666,-4.004819,0.726640,3.644306,...,3.349453,-1.084766,0.639891,1.337987,1.886551,2.040363,2.076126,-0.199317,2.993108,-4.508162
